In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
## Load the ANN Trained Model, Scalar, OHE, Label Encoder
model = load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
## Example Input Data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
## One Hot Encode 'Geography'
geo_encoded = ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/ziko/Documents/Python/Udemy/Krish Naik/Neural Networks/ANN Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_data = pd.DataFrame([input_data])

In [6]:
## Combine One Hot Encoded Data with input data
input_data = pd.concat([input_data.reset_index(drop = True), geo_encoded_df], axis=1)

In [7]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])

In [8]:
input_data.drop('Geography', axis=1, inplace=True)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
## Scaling the Data
input_data = scaler.transform(input_data)

In [13]:
## Predict
prediction = model.predict(input_data)
prediction_prob = prediction[0][0]
print(prediction_prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.03720901


In [14]:
if (prediction_prob > 0.5):
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
